In [1]:
import pandas as pd
import os

La tabla que limita a las demás es la de las descripciones

In [2]:
df_limitante = pd.read_excel('DescripcionesEspecies.xlsx')

#### Cargamos el resto de tablas que hacen referencia a especies, y eliminamos aquellas que no esten en la tabla anterior

Tabla de taxonomía

In [3]:
df_taxonomia = pd.read_excel('../1. Obtención Y Procesado de Datos/EIDOS_taxonomia.xlsx')
df_taxonomia = df_taxonomia[df_taxonomia['taxonid'].isin(df_limitante['idtaxon'])][[
    'taxonid', 'name', 'taxonrank', 'origen', 'taxonomicgroup', 'kingdom',
       'phylum', 'class', 'order', 'family', 'genus', 'subgenus',
       'specificepithet', 'infraspecificepithet', 'nametype'
]]
# Añadimos una nueva columna con el estado legal de las especies (si es invasora o no)
df_estado_legal = pd.read_excel('../1. Obtención Y Procesado de Datos/Estado legal especies/EIDOS_estado_legal.xlsx')
df_taxonomia = df_taxonomia.merge(df_estado_legal[['idtaxon']], left_on='taxonid', right_on='idtaxon', how='left')
df_taxonomia['isinvasive'] = df_taxonomia['idtaxon'].apply(lambda x: 'Invasora' if pd.notnull(x) else 'No invasora')
df_taxonomia.drop(columns=['idtaxon'], inplace=True)
# Añadimos una nueva columna con el estado de conservación más reciente de las especies
df_estado_conservacion = pd.read_excel('../1. Obtención Y Procesado de Datos/Estado conservación especies/EIDOS_estado_conservacion.xlsx')
df_taxonomia = df_taxonomia.merge(
df_estado_conservacion[['idtaxon', 'categoriaconservacion']], left_on='taxonid', right_on='idtaxon', how='left')
df_taxonomia.drop(columns=['idtaxon'], inplace=True)
df_taxonomia.rename(columns={"categoriaconservacion": "conservationstatus"}, inplace=True)
df_taxonomia['conservationstatus'] = df_taxonomia['conservationstatus'].str.replace("\"", "")
# Eliminar los rangos "Nombre mal aplicado" y "Nothospecies"
df_taxonomia = df_taxonomia[~df_taxonomia['taxonrank'].isin(['Nombre mal aplicado', 'Nothospecies'])]
# Poner los nombres aceptados como "Aceptado" y el taxon que solamente tiene un nombre ("Sinónimo ambiguo") como "Aceptado" también
df_taxonomia['nametype'] = df_taxonomia['nametype'].str.replace('Aceptado/válido', 'Aceptado').replace('Sinónimo ambiguo', 'Aceptado')
# Cambiar todos los demás por "Sinónimo" (solamente cambia 28 nombres)
df_taxonomia['nametype'] = df_taxonomia['nametype'].apply(lambda x: "Sinónimo" if x != "Aceptado" else x)
# Necesitamos renombrar la columna de taxonomia porque para SQL es una palabra reservada
df_taxonomia = df_taxonomia.rename(columns={'order': 'taxonorder'})
# Para que aparezcan los nombres aceptados primero
df_taxonomia = df_taxonomia.sort_values(by=['taxonid', 'nametype'])
df_taxonomia.to_excel('Taxonomia.xlsx', index=False)

Tabla de ubicaciones (cuadrículas) de especies

In [ ]:
df_ubicaciones = pd.read_csv('../1. Obtención Y Procesado de Datos/Ubicaciones y media/UBICACIONES_TODAS.csv')
df_ubicaciones = df_ubicaciones[df_ubicaciones['idtaxon'].isin(df_limitante['idtaxon'])]
df_ubicaciones = df_ubicaciones['cuadricula'] = df_ubicaciones['cuadricula'].str.strip()
df_ubicaciones = df_ubicaciones[df_ubicaciones['cuadricula'].str.len() == 7]
df_ubicaciones.to_csv('CuadriculasEspecies.csv', index=False)

Tabla de nombres de especies

In [ ]:
df_nombres = pd.read_excel('../1. Obtención Y Procesado de Datos/Nombres especies/EIDOS_nombres.xlsx')
# Aplicamos el filtro de descripciones
df_nombres = df_nombres[df_nombres['idtaxon'].isin(df_limitante['idtaxon'])]
df_nombres = df_nombres[['idtaxon', 'nombre_comun', 'idioma', 'espreferente']]
df_nombres = df_nombres.sort_values(by='espreferente', ascending=False)
df_nombres = df_nombres.drop_duplicates(subset=['idtaxon', 'nombre_comun', 'idioma'])
df_nombres = df_nombres.sort_values(by='idtaxon')
# Aquellos con un solo valor NA y lo demas 0, se le asigna el valor NA como preferente
sin_preferente = df_nombres[~df_nombres['idtaxon'].isin(df_nombres[(df_nombres['espreferente'] == 1) & (df_nombres['idioma'] == 'Castellano')]['idtaxon'])]
solo_un_nan_dict = df_nombres[df_nombres['espreferente'].isna() & (df_nombres['idioma'] == 'Castellano')]['idtaxon'].value_counts() == 1
solo_un_na_sin_preferente = sin_preferente[sin_preferente['idtaxon'].apply(lambda x: solo_un_nan_dict[x] if x in solo_un_nan_dict else False)]
indices_para_preferente = solo_un_na_sin_preferente[solo_un_na_sin_preferente['espreferente'].isna() & (solo_un_na_sin_preferente['idioma'] == 'Castellano')].index
df_nombres.loc[indices_para_preferente, 'espreferente'] = 1
# Aquellos que solo tienen un unico registro con un 0, se le asigna ese como preferente
sin_preferente = df_nombres[~df_nombres['idtaxon'].isin(df_nombres[(df_nombres['espreferente'] == 1) & (df_nombres['idioma'] == 'Castellano')]['idtaxon'])]
solo_uno_dict = sin_preferente[sin_preferente['idioma'] == 'Castellano']['idtaxon'].value_counts() == 1
solo_uno_sin_preferente = sin_preferente[sin_preferente['idtaxon'].apply(lambda x: solo_uno_dict[x] if x in solo_uno_dict else False) & (sin_preferente['idioma'] == 'Castellano')]
df_nombres.loc[solo_uno_sin_preferente.index, 'espreferente'] = 1
# Las que quedan sin nombre preferente, se extraen a un archivo para asignacion manual
sin_preferente = df_nombres[~df_nombres['idtaxon'].isin(df_nombres[(df_nombres['espreferente'] == 1) & (df_nombres['idioma'] == 'Castellano')]['idtaxon'])]
sin_preferente[sin_preferente['idioma'] == 'Castellano'].to_excel('Necesitan_Preferente.xlsx', index=False)
# Guardamos el resultado
df_nombres.to_csv('NombresEspecies.csv', index=False)

In [ ]:
# Una vez asignado, se unifican ambos archivos
df_nombres = pd.read_csv('NombresEspecies.csv')
df_nombres_a_actualizar = pd.read_excel('Necesitan_Preferente.xlsx')
df_nombres = pd.concat([df_nombres, df_nombres_a_actualizar])
df_nombres = df_nombres.drop_duplicates(subset=['idtaxon', 'nombre_comun', 'espreferente'], keep='last')
df_nombres.to_csv('NombresEspecies.csv', index=False)
os.remove('Necesitan_Preferente.xlsx')

También eliminamos columnas innecesarias de la tabla de Rutas

In [ ]:
df_rutas = pd.read_csv('../1. Obtención Y Procesado de Datos/Rutas/RUTAS_TODAS.csv')
df_rutas = df_rutas[['ID_Ruta', 'Nombre_Ruta', 'NOM_ETAPA', 'Longitud', 'Description', 'nombre_en', 'CCAA_1', 'Provincia_1']]
df_rutas = df_rutas.rename(columns={'nombre_en': 'Nombre_Ingles', 'CCAA_1': 'CCAA', 'Provincia_1': 'Provincia', 'NOM_ETAPA': 'Nombre_Etapa', 'Description': 'Descripcion'})

def clean_description(d):
    if not isinstance(d, str):
       d = ''
    else:
       d = d[d.find('Descripción'):].split('<td>')[1].split('</')[0]
    return d

df_rutas['Descripcion'] = df_rutas['Descripcion'].apply(clean_description)
df_rutas['Longitud'] = df_rutas['Longitud'].str.replace(',', '.').astype(float)
df_rutas.to_excel('Rutas.xlsx', index=False)

Y quitamos los espacios en blanco de la tabla de CuadriculasRutas

In [ ]:
df_cuadriculas_rutas = pd.read_csv('../1. Obtención Y Procesado de Datos/Rutas/Rutas_Cuadriculas/Rutas_Cuadriculas.csv')
df_cuadriculas_rutas['CUADRICULA'] = df_cuadriculas_rutas['CUADRICULA'].str.strip()
df_cuadriculas_rutas.to_csv('CuadriculasRutas.csv', index=False)

Y arreglamos la consistencia de la tabla Cuadriculas

In [ ]:
# Debemos asegurarnos de que la Comunidad Autonoma es consistente con la provincia
df_cuadriculas = pd.read_csv('../1. Obtención Y Procesado de Datos/mallas_con_ccaa_y_prov.csv')
# Para ello, obtenemos el diccionario de provincia a Comunidad Autonoma viendo las cuadriculas que solo aparecen una vez
solo_una_vez = df_cuadriculas['CUADRICULA'].value_counts() == 1
cuadriculas_solo_una_vez = df_cuadriculas[df_cuadriculas['CUADRICULA'].apply(lambda x: solo_una_vez[x])]
ccaas = {row['Provincia']: row['CCAA'] for i, row in cuadriculas_solo_una_vez.iterrows()}
df_cuadriculas['CCAA'] = df_cuadriculas['Provincia'].apply(lambda x: ccaas[x])
df_cuadriculas = df_cuadriculas.drop_duplicates()
df_cuadriculas.to_csv('Cuadriculas.csv', index=False)